<a href="https://colab.research.google.com/github/clonedSemicolon/Crash-Predictor/blob/master/Crash_Prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.tree import plot_tree

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import folium

In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/merged_dataset.csv'
df = pd.read_csv(path)
df.info()


<ipython-input-18-4ee6aa74ce99>:3: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977117 entries, 0 to 2977116
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE               object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  DAMAGE                   object 
 13  PRIM_CONTRIBUTORY_CAUSE  object 
 14  BEAT_OF_OCCURRENCE       float64
 15  NUM_UNITS                int64  
 16  INJURIES_TOTAL           float64
 17  CRASH_HOUR               int64  
 18  CRASH_DAY_OF_WEEK        object 
 19  CRASH_MONTH              int64  
 20  LATITUDE                 float64
 21  LONGITUD

In [ ]:
# calculating nulls
nulls = df.isna().sum()
nulls

,0
CRASH_RECORD_ID,0
CRASH_DATE,0
POSTED_SPEED_LIMIT,0
TRAFFIC_CONTROL_DEVICE,0
DEVICE_CONDITION,0
WEATHER_CONDITION,0
LIGHTING_CONDITION,0
FIRST_CRASH_TYPE,0
TRAFFICWAY_TYPE,0
ALIGNMENT,0


In [ ]:
df.fillna(0, inplace=True)
df.dropna(axis = 1, inplace =True)
full_dataset = df.rename(columns=str.lower)
full_dataset

,crash_record_id,crash_date,posted_speed_limit,traffic_control_device,device_condition,weather_condition,lighting_condition,first_crash_type,trafficway_type,alignment,...,age,drivers_license_class,safety_equipment,airbag_deployed,injury_classification,ems_agency,driver_vision,bac_result,bac_result value,cell_phone_use
0,00a530520c84927490b597a6220ff3f2a3347472ef3086...,2025-01-15 13:50:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,RAMP,STRAIGHT AND LEVEL,...,55.0,A,SAFETY BELT USED,NOT APPLICABLE,NO INDICATION OF INJURY,0,NOT OBSCURED,TEST NOT OFFERED,0.0,0
1,204c3ca48ff3cdf0737e7f10b54ef5f8c49fbe30b5e53e...,2024-12-04 17:39:00,30,UNKNOWN,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,32.0,D,USAGE UNKNOWN,DID NOT DEPLOY,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
2,204c3ca48ff3cdf0737e7f10b54ef5f8c49fbe30b5e53e...,2024-12-04 17:39:00,30,UNKNOWN,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,26.0,D,USAGE UNKNOWN,"DEPLOYED, COMBINATION",NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
3,204c3ca48ff3cdf0737e7f10b54ef5f8c49fbe30b5e53e...,2024-12-04 17:39:00,30,UNKNOWN,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,32.0,D,USAGE UNKNOWN,DID NOT DEPLOY,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
4,204c3ca48ff3cdf0737e7f10b54ef5f8c49fbe30b5e53e...,2024-12-04 17:39:00,30,UNKNOWN,UNKNOWN,RAIN,"DARKNESS, LIGHTED ROAD",TURNING,NOT DIVIDED,STRAIGHT AND LEVEL,...,26.0,D,USAGE UNKNOWN,"DEPLOYED, COMBINATION",NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2977112,bfa8377168b8dc75b4a09bf627b4e28fc77f465911c99b...,2023-09-13 13:08:00,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,31.0,D,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
2977113,bfa8377168b8dc75b4a09bf627b4e28fc77f465911c99b...,2023-09-13 13:08:00,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,27.0,D,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
2977114,bfa8377168b8dc75b4a09bf627b4e28fc77f465911c99b...,2023-09-13 13:08:00,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,31.0,D,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0
2977115,bfa8377168b8dc75b4a09bf627b4e28fc77f465911c99b...,2023-09-13 13:08:00,30,UNKNOWN,UNKNOWN,UNKNOWN,UNKNOWN,TURNING,FOUR WAY,STRAIGHT AND LEVEL,...,27.0,D,USAGE UNKNOWN,DEPLOYMENT UNKNOWN,NO INDICATION OF INJURY,0,UNKNOWN,TEST NOT OFFERED,0.0,0


In [ ]:
# Define risk factor clearly based on injuries_total:
def risk_category(injuries):
    if injuries >= 3:
        return 'High'
    elif injuries >= 1:
        return 'Medium'
    else:
        return 'Low'

df['risk_factor'] = df['INJURIES_TOTAL'].apply(risk_category)

# Drop columns not useful for modeling or redundant after creating target variable
df.drop(['INJURIES_TOTAL', 'LOCATION'], axis=1, inplace=True)


In [ ]:
# Convert crash_date into datetime and extract features
df['crash_date'] = pd.to_datetime(df['CRASH_DATE'])
df['crash_day'] = df['crash_date'].dt.day
df['crash_year'] = df['crash_date'].dt.year

# Drop original crash_date after extraction
df.drop('CRASH_DATE', axis=1, inplace=True)

In [ ]:
X = df.drop('risk_factor', axis=1)  # Features (all except target)
y = df['risk_factor']

In [ ]:
y

,risk_factor
0,Low
1,Low
2,Low
3,Low
4,Low
...,...
2977112,Low
2977113,Low
2977114,Low
2977115,Low


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Identify numeric and categorical columns explicitly again after dropping columns above.
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_cols, numeric_cols),
        ('cat', categorical_cols, categorical_cols)
    ])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([('imputer_num', SimpleImputer(strategy='median')),
                          ('scaler', StandardScaler())]), numeric_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])


In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

# Train the pipeline (this will fix your KeyError issue!)
model_pipeline.fit(X_train, y_train)


TypeError: Encoders require their input argument must be uniformly strings or numbers. Got ['int', 'str']

In [ ]:
print(df.shape)
df.info()

(2977117, 41)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2977117 entries, 0 to 2977116
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CRASH_RECORD_ID          object 
 1   CRASH_DATE               object 
 2   POSTED_SPEED_LIMIT       int64  
 3   TRAFFIC_CONTROL_DEVICE   object 
 4   DEVICE_CONDITION         object 
 5   WEATHER_CONDITION        object 
 6   LIGHTING_CONDITION       object 
 7   FIRST_CRASH_TYPE         object 
 8   TRAFFICWAY_TYPE          object 
 9   ALIGNMENT                object 
 10  ROADWAY_SURFACE_COND     object 
 11  ROAD_DEFECT              object 
 12  DAMAGE                   object 
 13  PRIM_CONTRIBUTORY_CAUSE  object 
 14  BEAT_OF_OCCURRENCE       float64
 15  NUM_UNITS                int64  
 16  INJURIES_TOTAL           float64
 17  CRASH_HOUR               int64  
 18  CRASH_DAY_OF_WEEK        object 
 19  CRASH_MONTH              int64  
 20  LATITUDE                 float64

In [ ]:
def risk_category(injuries):
    if injuries >= 3:
        return 'High'
    elif injuries >= 1:
        return 'Medium'
    else:
        return 'Low'

df['risk_factor'] = df['INJURIES_TOTAL'].apply(risk_category)

# Drop unnecessary columns after creating target variable
df.drop(['INJURIES_TOTAL', 'CRASH_RECORD_ID', 'LOCATION'], axis=1, inplace=True)

In [ ]:
# Convert crash_date to datetime and extract relevant features
df['crash_date'] = pd.to_datetime(df['CRASH_DATE'])
df['crash_day'] = df['crash_date'].dt.day
df['crash_year'] = df['crash_date'].dt.year

# Drop original crash_date column after extraction
df.drop('CRASH_DATE', axis=1, inplace=True)

In [ ]:
# Separate numeric and categorical columns for different imputation strategies
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Numeric data imputation (median)
for col in numeric_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Categorical data imputation (most frequent)
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


<ipython-input-12-f80183a89247>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
<ipython-input-12-f80183a89247>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [ ]:
X = df.drop('risk_factor', axis=1)
y = df['risk_factor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from xgboost import XGBClassifier

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numeric_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])


In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42))
])

# Train the model pipeline on training data
model_pipeline.fit(X_train, y_train)

ValueError: A given column is not a column of the dataframe